## Python Module: Reserve a Server 

This notebook demonstrates how to use the OpenStack python API to create a reservation for a server.

Chameleon requires most resources to be reserved before they can be used. Reservations can not be created independently. Instead, all reservatons must be part of a *Lease*.  Leases are composed of a group of one or more reservations that, typically, include all reservations required for an experiment.  

#### Related Modules

- [Get Lease](../../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../../modules-python/reservations/get_reservation_from_lease.ipynb)

#### Import Library

```
import chi
from chi.reservation_api_examples import *
```

#### Arguments

Required:

- `lease_name`: The name of the targeted lease

Optional:

- `node_type`: The type of server rquested. Default: 'compute_haswell'. Vaild values: **TODO: insert nodes types**
- `count`: The number of requsted servers. Default: 1.

#### Imported Methods used by Example(s) 

#### OpenStack API Method 

Note: The following code i

In [ ]:
from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = "%Y-%m-%d %H:%M"

def add_node_reservation(res, count=1, node_type="compute_haswell"):
    res.append({
        "resource_type": "physical:host",
        "resource_properties": json.dumps(["==", "$node_type", node_type]),
        "hypervisor_properties": "",
        "min": count,
        "max": count
    })

def reserve_node(lease_name,node_type="compute_haswell",count=1):
    # Set start/end date for lease
    # Start one minute into future to avoid Blazar thinking lease is in past
    # due to rounding to closest minute.
    start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
    end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)
    
    # Build list of reservations (in this case there is only one reservation)
    reservation_list = []
    add_node_reservation(reservation_list, count=count, node_type=node_type)

    # Create the lease
    lease = chi.blazar().lease.create(name=lease_name, 
                                start=start_date,
                                end=end_date,
                                reservations=reservation_list, events=[])

## Examples: 
**Important**: Examples below import API methods from chi library and does not use refernece code above!

#### Reserve a Node

In [ ]:
import json
import os
import chi

from chi.reservation_api_examples import *

#Config with your project and site
chi.set('project_name', 'CH-816532') # Replace with your project name
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

#Create the lease
reserve_node("myNewLeaseName", node_type="compute_haswell")

#Get the lease by name
lease = get_lease_by_name("myNewLeaseName")
    
#Print the lease info
print(json.dumps(lease, indent=2))
